# Read and Explore Data

In [4]:
import sys

sys.path.append("../")

import numpy as np
import pandas as pd
from typing import Union, Tuple
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [5]:
from task1.retrieval_system import RetrievalSystem, SongInfo
from task1.similarity_measure import (
    cosine_similarity,
    dot_product,
    manhattan_distance,
    euclidean_distance,
    random_similarity,
)
from utils import read, embed_and_merge

In [6]:
# basic song information from task 1
df = read("information", 0)
df

,id,artist,song,album_name
0,01Yfj2T3YTwJ1Yfy,We As Human,Take The Bullets Away (feat. Lacey Sturm),We As Human
1,01gyRHLquwXDlhkO,The Notorious B.I.G.,Somebody's Gotta Die,Life After Death (Remastered Edition)
2,01rMxQv6vhyE1oQX,Against the Current,Chasing Ghosts,In Our Bones
3,02RGE9FNH65RtMS7,Barthezz,Infected,Trance - The Early Years (1997-2002)
4,02ZnlCGZEbkfCDxo,Laura Pausini,Tra Te E Il Mare,The Best of Laura Pausini - E Ritorno Da Te
...,...,...,...,...
10090,zyzILCQvVeUFIINi,Crowded House,When You Come,Temple Of Low Men
10091,zzgS4ZqyswamEWNj,Britney Spears,My Only Wish (This Year),Platinum Christmas
10092,zzoFYDMlqU1X2zz1,Thundercat,DUI,Drunk
10093,zzpkRCGA5ud8q4mv,Otis Redding,Rock Me Baby,Otis Blue


In [7]:
# add genre information for metric calculation
genres = read("genres", 0)
# convert genre to actual list via eval
genres["genre"] = genres["genre"].apply(eval).apply(set)
df = df.merge(genres, on="id", how="left")

We load one new feature and the features from the previous assignments. We need them for comparison and fusion.

In [8]:
visual_feature = "resnet"
stats = read(visual_feature, 0)
df = embed_and_merge(df, stats, visual_feature)

for audio_feature in ["mfcc_bow", "blf_spectral", "ivec256", "musicnn"]:
    stats = read(audio_feature, 0)
    df = embed_and_merge(df, stats, audio_feature)

for text_feature in ["lyrics_bert", "lyrics_word2vec", "lyrics_tf-idf"]:
    stats = read(text_feature, 0)
    df = embed_and_merge(df, stats, text_feature.split("_")[1])

In [9]:
df.isna().sum()

id              0
artist          0
song            0
album_name      0
genre           1
resnet          1
mfcc_bow        1
blf_spectral    1
ivec256         1
musicnn         1
bert            0
word2vec        0
tf-idf          0
dtype: int64

In [10]:
# data for task 2 does not include the item with id "03Oc9WeMEmyLLQbj" = row 5
df = df.drop(5)
df = df.reset_index()

# Define retrieval systems

## From Task 1 (text-based)

In [11]:
rs_random = RetrievalSystem(
    df=df,
    sim_metric=random_similarity,
)

In [12]:
rs_cos_tdidf = RetrievalSystem(
    df=df,
    sim_metric=cosine_similarity,
    sim_feature="tf-idf",
)

In [13]:
rs_cos_bert = RetrievalSystem(
    df=df,
    sim_metric=cosine_similarity,
    sim_feature="bert",
)

In [14]:
rs_dot_w2v = RetrievalSystem(
    df=df,
    sim_metric=dot_product,
    sim_feature="word2vec",
)

## From Task 2 (audio-based)

In [15]:
rs_cos_mfcc = RetrievalSystem(
    df=df,
    sim_metric=cosine_similarity,
    sim_feature="mfcc_bow",
)

In [16]:
rs_cos_blf = RetrievalSystem(
    df=df,
    sim_metric=cosine_similarity,
    sim_feature="blf_spectral",
)

In [17]:
rs_cos_ivec256 = RetrievalSystem(
    df=df,
    sim_metric=cosine_similarity,
    sim_feature="ivec256",
)

In [18]:
rs_cos_dnn = RetrievalSystem(
    df=df,
    sim_metric=cosine_similarity,
    sim_feature="musicnn",
)

## From Task 3 (video-based; new!)


In [19]:
rs_cos_resnet = RetrievalSystem(
    df=df,
    sim_metric=cosine_similarity,
    sim_feature="resnet",
)

In [24]:
# create pd dataframe consisting of all retrieval systems, with metric and feature
rs = pd.DataFrame(
    [
        ["random", "random", rs_random],
        ["cosine", "tf-idf", rs_cos_tdidf],
        ["cosine", "bert", rs_cos_bert],
        ["dot", "word2vec", rs_dot_w2v],
        ["cosine", "mfcc_bow", rs_cos_mfcc],
        ["cosine", "blf_spectral", rs_cos_blf],
        ["cosine", "ivec256", rs_cos_ivec256],
        ["cosine", "musicnn", rs_cos_dnn],
        ["cosine", "resnet", rs_cos_resnet],
    ],
    columns=["metric", "feature", "rs_object"],
)

In [25]:
rs

,metric,feature,rs_object
0,random,random,<task1.retrieval_system.RetrievalSystem object...
1,cosine,tf-idf,<task1.retrieval_system.RetrievalSystem object...
2,cosine,bert,<task1.retrieval_system.RetrievalSystem object...
3,dot,word2vec,<task1.retrieval_system.RetrievalSystem object...
4,cosine,mfcc_bow,<task1.retrieval_system.RetrievalSystem object...
5,cosine,blf_spectral,<task1.retrieval_system.RetrievalSystem object...
6,cosine,ivec256,<task1.retrieval_system.RetrievalSystem object...
7,cosine,musicnn,<task1.retrieval_system.RetrievalSystem object...
8,cosine,resnet,<task1.retrieval_system.RetrievalSystem object...


## Fusion Techniques

### Early Fusion

In [33]:
from sklearn.preprocessing import StandardScaler

# Select two features for early fusion (e.g., lyrics_tf-idf and mfcc_bow)
textual_feature = df['tf-idf']
audio_feature = df['mfcc_bow']

# Standardize the features
scaler = StandardScaler()
textual_feature = scaler.fit_transform(textual_feature.reshape(-1, 1))
audio_feature = scaler.fit_transform(audio_feature.reshape(-1, 1))

# Perform early fusion using a simple aggregation (e.g., average)
early_fusion_feature = (textual_feature + audio_feature) / 2

# Add the early fusion feature to the DataFrame
df['early_fusion'] = early_fusion_feature

# Now you can use the 'early_fusion' feature in retrieval similar to text-based(<similarity>, <feature>)


AttributeError: 'Series' object has no attribute 'reshape'

In [39]:
from sklearn.preprocessing import StandardScaler

# Auswahl der Merkmale
textual_feature = df['bert'].to_numpy()
audiovisual_feature = df['mfcc_bow'].to_numpy()  # oder df['resnet'].to_numpy() je nach Auswahl

# Vorbereitung der Daten und Merkmale
# ...

# Standardisierung der Merkmale
scaler_textual = StandardScaler()
scaler_audiovisual = StandardScaler()

textual_feature = scaler_textual.fit_transform(textual_feature.reshape(-1, 1))
audiovisual_feature = scaler_audiovisual.fit_transform(audiovisual_feature.reshape(-1, 1))

# Fusionstechnik: Einfacher Durchschnitt
early_fusion_feature = (textual_feature + audiovisual_feature) / 2

# Füge das fusionierte Merkmal zum DataFrame hinzu
df['early_fusion'] = early_fusion_feature

# Retrieval auf der Grundlage des fusionierten Merkmals
rs_cos_early_fusion = RetrievalSystem(
    df=df,
    sim_metric=cosine_similarity,
    sim_feature="early_fusion",
)


ValueError: setting an array element with a sequence.

In [41]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Example data
textual_feature = df['bert'].to_numpy()
audiovisual_feature = df['mfcc_bow'].to_numpy()

# Reshape the features to 2D arrays
textual_feature = textual_feature.reshape(-1, 1)
audiovisual_feature = audiovisual_feature.reshape(-1, 1)

# Concatenate the features along the columns
combined_features = np.concatenate((textual_feature, audiovisual_feature), axis=1)

# Apply PCA to reduce dimensionality
pca = PCA(n_components=1)
combined_features_pca = pca.fit_transform(combined_features)

# Initialize StandardScaler
scaler = StandardScaler()

# Fit and transform the PCA-transformed features
scaled_features = scaler.fit_transform(combined_features_pca)

# Now scaled_features contains the combined and scaled features


ValueError: setting an array element with a sequence.

In [34]:
sample_song = SongInfo(title="Zombie", artist="The Cranberries")
# query most be a pd.Series --> convert here
sample_query = df[
    (df["song"] == sample_song.title) & (df["artist"] == sample_song.artist)
].iloc[0]

In [35]:
sample_query.genre

{'alternative rock',
 'celtic',
 'celtic rock',
 'classic rock',
 'grunge',
 'hard rock',
 'indie rock',
 'irish rock',
 'metal',
 'pop',
 'pop rock',
 'post grunge',
 'rock',
 'soft rock'}

In [36]:
results = rs_cos_resnet.retrieve(sample_song, 10)

In [37]:
results

,index,id,artist,song,album_name,genre,resnet,mfcc_bow,blf_spectral,ivec256,musicnn,bert,word2vec,similarity
0,1350,87dM2jCka3uQ9jCe,Cannibal Corpse,Kill or Become,A Skeletal Domain,"{technical death metal, metal, brutal death me...","[3.362233, 2.601369, 1.600803, 0.060141, 0.095...","[0.0, 0.0, 2.3344537999999995, 0.0, 0.0, 0.0, ...","[-0.0240165, -0.0199949, -0.0172412, -0.014370...","[-0.4084688127040863, -1.7257150411605835, 0.2...","[0.10409315, 0.0003639398, 0.0018088311, 0.172...","[0.0218665041029453, 0.0542871952056884, 0.021...","[0.0381368775672069, 0.0218782743709233, 0.008...",0.916257
1,2383,EPq10w8WXMzn6SiM,Lamb of God,Overlord,VII: Sturm Und Drang (Deluxe),"{hard rock, alternative rock, deathcore, grind...","[1.898288, 2.382536, 1.860425, 0.11504, 0.1967...","[0.0, 0.0, 2.3926969, 0.0, 0.0, 0.0, 2.9159272...","[-0.0285346, -0.0240487, -0.0222086, -0.018877...","[-0.6786251068115234, -0.8794464468955994, 0.1...","[0.22721322, 0.0007293002, 0.01049833, 0.04474...","[0.0362034477293491, 0.0077602691017091, 0.001...","[0.0377602214862373, 0.0091797875074641, -0.00...",0.911563
2,3422,L3Kp1jbgQ610Ug6z,Madonna,Spanish Eyes,Like a Prayer,"{pop, latin, singer songwriter}","[2.155628, 2.735073, 1.582092, 0.15057, 0.0772...","[1.3222193000000002, 0.0, 1.770852, 0.0, 1.857...","[-0.031409, -0.0261219, -0.0239373, -0.0207116...","[0.9031068086624146, 0.3960944414138794, -0.27...","[0.31000128, 0.0033261061, 0.077820435, 0.0101...","[-0.0262783914804458, 0.0460927821695804, -0.0...","[0.023038339332325, 0.0050681515854808, 0.0258...",0.909842
3,4953,UOuWHCEIps7Mjz3E,Oasis,Shakermaker,Definitely Maybe (Remastered) [Deluxe Edition],"{soft rock, alternative rock, grunge, rock and...","[1.827026, 2.029464, 2.199537, 0.053277, 0.056...","[0.0, 0.0, 1.462398, 0.0, 0.0, 0.0, 2.1931245,...","[-0.0331521, -0.0284585, -0.0256427, -0.022688...","[-1.3665876388549805, -0.6717993021011353, -1....","[0.38178083, 0.0028633927, 0.046525437, 0.0209...","[0.0184765495359897, 0.0482282564043998, -0.02...","[0.0095084663467866, 0.0075772250553279, 0.013...",0.907669
4,7505,kGqt1SWYRG5M2X32,Paramore,Careful,brand new eyes,"{indie punk, alternative rock, pop rock, pop p...","[1.278383, 2.580097, 2.079799, 0.014902, 0.116...","[1.4771212, 0.0, 1.7160033, 0.0, 0.84509799999...","[-0.029709, -0.0261018, -0.0212901, -0.0193414...","[-0.5033568739891052, 0.167227953672409, -0.67...","[0.29173714, 0.0030574615, 0.01691651, 0.04722...","[0.0268798135221004, 0.0295519027858972, -0.01...","[0.0314850911620612, 0.0170346358741976, 0.016...",0.906508
5,7534,kSDPUwWDCOkb8wOX,Cutting Crew,(I Just) Died in Your Arms,The Best Of Cutting Crew,"{soft rock, pop rock, pop, classic rock, new w...","[0.856041, 1.370697, 1.149933, 0.017608, 0.055...","[1.7993406000000003, 0.0, 2.9836264, 0.0, 1.04...","[-0.0297599, -0.0242387, -0.0212726, -0.018415...","[-0.5980198383331299, 1.3942530155181885, -1.3...","[0.21125002, 0.017314926, 0.1454034, 0.0510449...","[0.0297185182571411, -0.0331318564713001, -0.0...","[0.0222849311511963, 0.0161594967748969, 0.025...",0.904957
6,2565,FakoHKQAGT6gFXP0,16 Horsepower,Haw,16 Horsepower,"{alternative rock, folk, gothic americana, alt...","[1.119987, 2.550595, 0.915632, 0.099561, 0.100...","[0.9542425, 0.0, 2.423246, 0.0, 0.69897, 0.0, ...","[-0.0291433, -0.0235362, -0.0200929, -0.018264...","[-0.7194744348526001, 1.293039083480835, -0.80...","[0.16388904, 0.011580788, 0.07119308, 0.013694...","[-0.0031567083206027, 0.013839672319591, -0.01...","[0.0319615606463179, 0.0277835686986721, 0.007...",0.904899
7,7806,m3SLg1agLGfpMdSz,Social Distortion,Ball and Chain,Social Distortion,"{classic rock, psychobilly, rock, hard rock, a...","[1.174189, 4.374656, 1.446732, 0.04104, 0.1384...","[0.0, 0.0, 1.8129133, 0.0, 0.0, 0.0, 2.6404815...","[-0.0322196, -0.0281729, -0.0251473, -0.022514...","[-1.1478033065795898, -0.4399683177471161, -0....","[0.26841068, 0.015795391, 0.039577536, 0.03961...","[0.0115926610305905, 0.066615305840969, -0.013...

### Late Fusion
In this section we will perform late fusion of 2 retrieval systems using score aggregation by:
- precomputing all retrievals and their similarities for chosen retrieval systems.
- checking statistical compatability of scores
- fusing systems via score average

# Evaluation